In [1]:
!git clone https://github.com/YUyou0/ict3104-team14-2023.git

Cloning into 'ict3104-team14-2023'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 68 (delta 4), reused 9 (delta 3), pack-reused 56
Receiving objects: 100% (68/68), 50.22 MiB | 45.63 MiB/s, done.
Resolving deltas: 100% (14/14), done.


# Select a dataset to be used for training

In [29]:
# Function to list file names in a folder and store their paths in an array
def list_file_names_and_paths(folder):
    global file_paths
    folder_path = os.path.join('/content/ict3104-team14-2023/input_videos', folder)
    if os.path.isdir(folder_path):
        file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]
        file_names = [os.path.basename(file) for file in file_paths]
        print(f"List of file names in '{folder}':")
        for file_name in file_names:
            print(file_name)
    else:
        print(f"'{folder}' is not a valid folder.")

# Create a dropdown widget with an empty default option
folder_dropdown = widgets.Dropdown(
    options=[''] + os.listdir('/content/ict3104-team14-2023/input_videos'),  # Include an empty option
    description='Select a Folder:',
)

# Initialize the file_paths array
file_paths = []

# Define an event handler for the dropdown
def on_dropdown_change(change):
    selected_folder = change.new
    list_file_names_and_paths(selected_folder)

# Attach the event handler to the dropdown
folder_dropdown.observe(on_dropdown_change, names='value')

# Display the dropdown widget
display(folder_dropdown)

Dropdown(description='Select a Folder:', options=('', 'charades'), value='')

List of file names in 'charades':
4GLAP.mp4
5O0YS.mp4
5D3X6.mp4
3JJ7C.mp4
2E8GU.mp4
3IPI3.mp4
8ASRQ.mp4
0RJKT.mp4
5EFQ7.mp4
52CKM.mp4
3LMTS.mp4


# List of videos to be used for training

In [31]:
print(file_paths)
print(len(file_paths))

['/content/ict3104-team14-2023/input_videos/charades/4GLAP.mp4', '/content/ict3104-team14-2023/input_videos/charades/5O0YS.mp4', '/content/ict3104-team14-2023/input_videos/charades/5D3X6.mp4', '/content/ict3104-team14-2023/input_videos/charades/3JJ7C.mp4', '/content/ict3104-team14-2023/input_videos/charades/2E8GU.mp4', '/content/ict3104-team14-2023/input_videos/charades/3IPI3.mp4', '/content/ict3104-team14-2023/input_videos/charades/8ASRQ.mp4', '/content/ict3104-team14-2023/input_videos/charades/0RJKT.mp4', '/content/ict3104-team14-2023/input_videos/charades/5EFQ7.mp4', '/content/ict3104-team14-2023/input_videos/charades/52CKM.mp4', '/content/ict3104-team14-2023/input_videos/charades/3LMTS.mp4']
11


# Input a name for this new model to be trained

In [38]:
# Create a text input widget
text_input = widgets.Text(
    value='',  # Default value (you can set a default value if needed)
    placeholder='Enter new model name',  # Placeholder text
    description='Name Input:',  # Label for the input field
)

# Create a submit button
submit_button = widgets.Button(
    description='Submit',  # Label for the button
)

# Create a variable to store the user's input
user_input_modelName = ''

# Define a function to update the user_input variable when the submit button is clicked
def on_submit_button_click(b):
    global user_input_modelName
    user_input_modelName = text_input.value
    if user_input_modelName == "":
      print("You have not created a name for this new model!")
    else:
      print("New model name has been submitted")
    if user_input_modelName.isalnum():
        print(f"Model name accepted: {user_input_modelName}")
    else:
        print("Invalid input. Please enter alphanumeric characters only (no spaces or special characters).")

# Attach the click event handler to the submit button
submit_button.on_click(on_submit_button_click)

# Display the widgets
display(text_input, submit_button)

Text(value='', description='Name Input:', placeholder='Enter new model name')

Button(description='Submit', style=ButtonStyle())

New model name has been submitted
Invalid input. Please enter alphanumeric characters only (no spaces or special characters).
New model name has been submitted
Model name accepted: naruto
New model name has been submitted
Model name accepted: naruto3


In [40]:
print(user_input_modelName)

naruto3
